# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [32]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

data_path = '/home/osboxes/Documents/data_set/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [33]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
OB_features = []
NUM_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        OB_features.append(feature)
    elif dtype == 'float64' or dtype == 'int64':
        NUM_features.append(feature)
print(OB_features)
print(NUM_features)

['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']


In [34]:
df['Name'] = df['Name'].fillna('None')
mean_df = df.groupby(['Name'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Name'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Name'])['Fare'].median().reset_index()
max_df = df.groupby(['Name'])['Fare'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Name'])
temp = pd.merge(temp, median_df, how='left', on=['Name'])
temp = pd.merge(temp, max_df, how='left', on=['Name'])
temp.columns = ['Name', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max']
temp

,Name,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,"Abbing, Mr. Anthony",7.5500,7.5500,7.5500,7.5500
1,"Abbott, Mr. Rossmore Edward",20.2500,20.2500,20.2500,20.2500
2,"Abbott, Mrs. Stanton (Rosa Hunt)",20.2500,20.2500,20.2500,20.2500
3,"Abelson, Mr. Samuel",24.0000,24.0000,24.0000,24.0000
4,"Abelson, Mrs. Samuel (Hannah Wizosky)",24.0000,24.0000,24.0000,24.0000
...,...,...,...,...,...
886,"de Mulder, Mr. Theodore",9.5000,9.5000,9.5000,9.5000
887,"de Pelsmaeker, Mr. Alfons",9.5000,9.5000,9.5000,9.5000
888,"del Carlo, Mr. Sebastiano",27.7208,27.7208,27.7208,27.7208
889,"van Billiard, Mr. Austin Blyler",14.5000,14.5000,14.5000,14.5000


In [35]:
df = pd.merge(df, temp, how='left', on=['Name'])
df = df.drop(['Name'] , axis=1)
df.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,7.2500,7.2500,7.2500,7.2500
1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,71.2833,71.2833,71.2833,71.2833
2,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,7.9250,7.9250,7.9250,7.9250
3,1,female,35.0,1,0,113803,53.1000,C123,S,53.1000,53.1000,53.1000,53.1000
4,3,male,35.0,0,0,373450,8.0500,NaN,S,8.0500,8.0500,8.0500,8.0500


In [36]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max']



,Pclass,Age,SibSp,Parch,Fare,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,3,22.0,1,0,7.2500,7.2500,7.2500,7.2500,7.2500
1,1,38.0,1,0,71.2833,71.2833,71.2833,71.2833,71.2833
2,3,26.0,0,0,7.9250,7.9250,7.9250,7.9250,7.9250
3,1,35.0,1,0,53.1000,53.1000,53.1000,53.1000,53.1000
4,3,35.0,0,0,8.0500,8.0500,8.0500,8.0500,8.0500


In [37]:
# 新特徵 + 線性迴歸 : 有些微改善
estimator = LinearRegression()
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.11619613893016814

In [38]:
# 新特徵 + 隨機森林
estimator = RandomForestRegressor()
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.019797277294230732

# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [40]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
# 沒有這四個新特徵的 dataframe 稱為 df_minus
df_minus = df.drop(['Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max'] , axis=1)

# 原始特徵 + 線性迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7038635542329971

In [41]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7005053927832138